# 02: Understanding the data

This notebook contains all of the information necessary for understanding the data and plotting light curves. In the following notebook I will explain the process of fitting light curves to the data.

In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy.stats import norm
np.random.seed(42)
%matplotlib inline  

## Downloading the data

We take 2 different types of data: LINEAR data and ZTF data. We download both from libraries in Python. 

### A bit about LINEAR

bla bla bla (read research papers)

### A bit about ZTF

bla bla bla(read research paper)

### Download process: LINEAR

We import all of the necessary libraries in order to download the LINEAR data

In [4]:
from astroML.utils.decorators import pickle_results
from astroML.time_series import search_frequencies, lomb_scargle, MultiTermFit
from astroML.datasets import fetch_LINEAR_sample
import numpy as np
import pandas as pd

In [2]:
data = fetch_LINEAR_sample(data_home='/Users/emadonev/Library/Mobile Documents/com~apple~CloudDocs/PROJECTS/var_stars') # fetching the data from astroML data library

[=========================================]  303.4kb / 303.4kb   
[=========================================]  16.05Mb / 16.05Mb   


Explain why is it like this that we are accessing the data...

In [20]:
c = 0
for i in data.ids:
    light_curve = data.get_light_curve(i)
    c += 2
    print(light_curve[:2])
print(c)

[[5.26514326e+04 1.57560000e+01 3.50000000e-02]
 [5.26514478e+04 1.57750000e+01 3.30000000e-02]]
[[5.26524776e+04 1.65340000e+01 5.90000000e-02]
 [5.26524924e+04 1.66480000e+01 6.40000000e-02]]
[[5.26374598e+04 1.54500000e+01 4.48000000e-01]
 [5.26524566e+04 1.54000000e+01 1.80000000e-02]]
[[5.26524564e+04 1.48020000e+01 1.10000000e-02]
 [5.26524716e+04 1.48010000e+01 1.10000000e-02]]
[[5.26524570e+04 1.38490000e+01 6.00000000e-03]
 [5.26524722e+04 1.38090000e+01 6.00000000e-03]]
[[5.26524571e+04 1.57110000e+01 2.60000000e-02]
 [5.26524723e+04 1.57250000e+01 2.60000000e-02]]
[[5.26644199e+04 1.60240000e+01 8.90000000e-02]
 [5.27822170e+04 1.60180000e+01 1.26000000e-01]]
[[5.26534613e+04 1.54790000e+01 2.00000000e-02]
 [5.26534766e+04 1.54890000e+01 2.20000000e-02]]
[[5.26534613e+04 1.50940000e+01 1.30000000e-02]
 [5.26534766e+04 1.51550000e+01 1.50000000e-02]]
[[5.26534612e+04 1.57050000e+01 2.40000000e-02]
 [5.26534765e+04 1.53540000e+01 1.80000000e-02]]
[[5.26534612e+04 1.60910000e+0

Our data contains 3 columns: `time`, `magnitude` and `magnitude_error`. In order to create lists with these values for every light curve, we use `.T` or we **transpose** the dataset. 

In [ ]:
time, mag, mag_er = data[ids[0]].T

### Download process: ZTF

In [ ]:
from ztfquery import lightcurve

In [ ]:
def getZTFlightcurve(ra, dec, radius=3.0):
    # matching radius is given in arcsec
    try:
       lcq = lightcurve.LCQuery()
       res = lcq.from_position(ra, dec, radius)
       ZTFdata = res.data # this is basically a DataFrame then
       # M. Graham recommends to get rid of obvious spurious points
       ZTFdata = ZTFdata.loc[ZTFdata['catflags'] < 32768]
    except Exception as e:
       print(e)
    return ZTFdata